## Graph inference code

**Warning** Most of this code is experimental. To be able to apply the graph inference process, the data is heavily subsampled, and therefore much of the information is lost. Because of this, the script main need to be run several times with different subsamplings before reaching coherent results. Moreover, the clustering techniques that have been applied are 'ad hoc' and can definetly be improved.

In [ ]:
# basic
import os
import time
import itertools  
import numpy as np
from difflib import SequenceMatcher
from collections import defaultdict

# machine learning
from scipy.linalg import block_diag
from scipy import linalg
from scipy import signal
import scipy.fftpack
from scipy.optimize import linear_sum_assignment
from scipy.sparse import csr_matrix, lil_matrix
from scipy.sparse.linalg import expm
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import peakutils

# plot
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex, colorConverter
hsv = plt.get_cmap('hsv')
from tqdm import tqdm

# pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

# custom
import video_loader

# multi-threading
from multiprocessing.dummy import Pool as ThreadPool 

In [ ]:
classes =  {0:'anger', 1:'contempt', 2:'disgust', 3:'fear', 4:'happy', 5:'sadness', 6:'surprise'}
N_frames = 20
N_landmarks = 68 
N_classes = len(classes)

# preprocessing
data_dir = os.path.join('/home','nii','Documents','CK+')

data_transforms = transforms.Compose(
    [transforms.Resize((64,64))])

K = 10
k_folders = ['set_' + str(idx) for idx in range(K)]   

    
training_datasets = {x: video_loader.VideoFolder(root=data_dir, image_folder='cohn-kanade-images-crop', 
                                 label_folder='Emotion', landmark_folder='Landmarks_crop',
                                 fold=x, phase='train', 
                                 classes=classes, n_frames=N_frames, n_landmarks=N_landmarks,
                                 transform=data_transforms, indexing=1)
                    for x in k_folders}

validation_datasets = {x: video_loader.VideoFolder(root=data_dir, image_folder='cohn-kanade-images-crop', 
                                 label_folder='Emotion', landmark_folder='Landmarks_crop',
                                 fold=x, phase='valid', 
                                 classes=classes, n_frames=N_frames, n_landmarks=N_landmarks,
                                 transform=data_transforms, indexing=1)
                    for x in k_folders}

In [ ]:
batch_size = 64

folds = [x for x in range(K)]

dataset = torch.utils.data.ConcatDataset([validation_datasets[k_folders[k]] 
                                                          for k in folds])

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True,
                                              num_workers=4)

dataset_size = len(dataset)

In [ ]:
videos, classes, land = next(iter(dataloader))
#landmarks = land.view(batch_size,N_frames,2*N_landmarks).permute(0,2,1)#.contiguous().view(2*N_landmarks, -1)
Y = land.view(batch_size,N_frames,2*N_landmarks).permute(2,1,0).contiguous()

In [ ]:
def get_landmarks():
    y_list = []
    for data in dataloader:
        _, _, land = data
        batch_size = land.size(0)
        land = land.view(batch_size, N_frames, 2*N_landmarks).permute(2,0,1).contiguous().view(2*N_landmarks, -1)
        y_list.append(land)
        
    return torch.cat(y_list, dim=1)

In [ ]:
Y = get_landmarks()

In [ ]:
Y.size()

In [ ]:
mask = np.ones((Y.size(0), 1),  dtype=bool)
mask[:34] = False
mask[120:122] = False
mask[128:130] = False
Y = Y[np.where(mask)[0],:]
N_landmarks = 49
Y.size()

In [ ]:
plt.plot(Y.numpy()[63,:])

In [ ]:
Ym = np.array(Y.numpy())
for i in range(2*N_landmarks):
    signal = Ym[i,:]
    yf = scipy.fftpack.fft(signal)
    peakind = peakutils.indexes(yf, thres=0.02/max(yf), min_dist=100)
    yf[peakind] = 0
    Ym[i,:] = np.real(scipy.fftpack.ifft(yf))

In [ ]:
fft_sig = abs(scipy.fftpack.fftshift(scipy.fftpack.fft(Y.numpy().reshape(2*N_landmarks, -1, order='F')[63,:])))
plt.plot(np.array(range(len(fft_sig))) - int(len(fft_sig)/2), fft_sig)

In [ ]:
peakind = peakutils.indexes(fft_sig, thres=0.02/max(fft_sig), min_dist=100)
fft_filt = np.array(fft_sig)
fft_filt[peakind] = 0
plt.plot(np.array(range(len(fft_filt))) - int(len(fft_filt)/2), fft_filt)

In [ ]:
def compute_kernel_maps(Y, N, T, L):
    l_list = []
    d_list = []

    # for all orders
    for l in range(L):
        k_list = []
        # for all landmarks
        for i in range(N):
            y = Y[i,:]
            sigma = np.std(y)
            K_t = np.zeros((T, T))
            # at all time steps
            for t in range(T):
                for tau in range(T):
                    # landmarks that are poorly detected
                    if sigma != 0:
                        shift = tau - l
                        if not (shift < 0 or shift >= T):
                            # compute kernel
                            if shift != t:
                                #K_t[t, tau] = np.exp(-abs(y[t]-y[shift])**2 / (2*sigma**2))
                                K_t[t, tau] = (y[t]*y[shift] + 1)**2

            k_list.append(K_t)

        K_l = np.concatenate((k_list), 1)
        D_l = block_diag(*k_list)

        l_list.append(K_l)
        d_list.append(D_l)

    K = np.concatenate((l_list), 1)
    D = block_diag(*d_list)
    
    return K, D

In [ ]:
Y = torch.DoubleTensor(Ym)
slices = int(Y.size(1)*0.004) 
Yf = Y.unfold(1, slices, 1)[:,0].numpy()

In [ ]:
# kernel map
N = N_landmarks * 2
T = Yf.shape[1]
L = 2

K, D = compute_kernel_maps(Yf, N, T, L)

In [ ]:
plt.plot(Yf[63])

In [ ]:
D = lil_matrix(D)

In [ ]:
plt.spy(D)

In [ ]:
# initialize
Gamma = np.zeros((N*T*L, N))
Xi = np.zeros((N*T*L, N))
k = 0

Kn = {}
Dn = {}

# filtered kernel maps
for j in range(0, N):
    Ij = np.array(range(j*T , (j+1)*T))
    Ijk = np.array([x for x in np.array(range(0 , N*T)) if x not in Ij])
    mask_k = np.ones(N*T, dtype=bool)
    mask_k[Ijk] = False
    
    Ijd = np.array(Ijk)
    for m in range(1, L):
        Ijd = np.concatenate((Ijd, Ijk + m*N*T))
        
    mask_d = np.ones(N*T*L, dtype=bool)
    mask_d[Ijd] = False
    
    #Kj = K[:, Ijk]
    Kj = np.array(K)
    Kj[:, mask_d] = 0
    Kn[j] = Kj
    
    #Dj = D[Ijk, :]
    #Dj = Dj[:, Ijd]
    Dj = lil_matrix(D)
    Dj[mask_d, :] = 0
    Dj[:, mask_d] = 0
    Dn[j] = Dj

In [ ]:
Ki = torch.from_numpy(K**(0.5))

In [ ]:
def block_shrinkage(z, lamb):
    return torch.clamp(z - lamb, min=0) / (z.norm() + 1e-6)
    #return z * torch.max(z.norm() - lamb, 0) / (z.norm() + 1e-6)

In [ ]:
def parallel(j, rho, lamb):
    # fetch variables
    Dj = torch.from_numpy(Dn[j].toarray())
    Kj = torch.from_numpy(Kn[j])
    Ki = torch.from_numpy(K**(0.5))
    xi = torch.from_numpy(Xi[:,j])
    y = torch.from_numpy(Yf[j,:])
    gamma = torch.from_numpy(Gamma[:,j])
    
    if use_gpu:
        Dj = Dj.cuda()
        Kj = Kj.cuda()
        Ki = Ki.cuda()
        xi = xi.cuda()
        y = y.cuda()
        gamma = gamma.cuda()
        
    # first step
    q = rho * Dj**0.5 @ gamma + Kj.transpose(1,0) @ y - Dj**0.5 @ xi
    
    # prevent matrix from being singular
    X = Kj.transpose(1,0) @ Kj + rho * Dj
    
    #X[np.diag_indices_from(X)] += 1e-4
    v = torch.diag(X) + 1e-4
    mask = torch.diag(torch.ones_like(v))
    X = mask*torch.diag(v) + (1. - mask)*X
    
    # second step
    #alpha = np.linalg.inv(X) @ q
    #X_LU = torch.btrifact(X)
    #alpha = torch.btrisolve(q, *X_LU)
    alpha = X.inverse() @ q
    
    # third step
    for i in range(N):
        for l in range(1,L):
            index = list(range(i*T+N*l, (i+1)*T+N*l))
            gamma[index] = block_shrinkage(Ki[:,index] @ alpha[index].transpose(-1,0) + xi[index]/rho, lamb)
            
    return alpha, gamma

In [ ]:
# initialize
use_gpu = True
Gamma = np.zeros((N*T*L, N))
Xi = np.zeros((N*T*L, N))
W = np.zeros((N*T*L, N))
k = 0

# hyper-parameters
rho = 1e-3
lamb = 0.01

start_time = time.time()
for k in range(3):
    for j in range(0, N):
        print('landmark ' + str(j))
        alpha, gamma = parallel(j, rho, lamb)
        W[:,j] = alpha
        Gamma[:,j] = gamma

    Xi += rho * (D.power(0.5) @ W - Gamma)
    
    print('Norm of W is : ' + str(np.linalg.norm(W)))

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
plt.figure(figsize=(2,30))
plt.imshow(W)

In [ ]:
W_0 = W[:N*T,:]
W_0 = abs(W_0.reshape(T,N,N)).mean(axis=0)
W_0[W_0 < 0.00000001] = 0
np.fill_diagonal(W_0, 0)
plt.imshow(W_0)

In [ ]:
W_1 = W[N*T:N*T*L,:]
W_1 = abs(W_1.reshape(T,N,N)).mean(axis=0)
W_1[W_1 < 0.000001] = 0
np.fill_diagonal(W_1, 0)
#W_1 = (W_1.T + W_1)/2
plt.imshow(W_1)

In [ ]:
weights = np.array(W_1)
# bibliometric symmetrization
U_d = weights @ weights.T + weights.T @ weights

In [ ]:
# degree-discounted symmetrization
weights = np.array(W_1)
d_o = np.sum(weights,axis=0) 
d_o[d_o == 0] = 1
D_o = np.diag(d_o**-0.5)

d_i = np.sum(weights,axis=1) 
d_i[d_i == 0] = 1
D_i = np.diag(d_i**-0.5)

B_d = D_o @ weights @ D_i @ weights.T @ D_o
C_d = D_i @ weights.T @ D_o @ weights @ D_i
U_d = B_d + C_d

In [ ]:
plt.imshow(U_d)

In [ ]:
weights = np.array(U_d)
degrees = np.sum(weights,axis=0)
degrees[degrees == 0] = 1
laplacian = np.diag(degrees**-0.5) @ (np.diag(degrees) - weights) @ np.diag(degrees**-0.5)
eigenvalues, eigenvectors = linalg.eigh(laplacian)
plt.plot(eigenvalues[:20], '.-', markersize=15);

In [ ]:
plt.plot(eigenvalues[2:20]/eigenvalues[1:19])

In [ ]:
Face = np.load('face_positions.npy')
mask = np.ones((Face.shape[0], ),  dtype=bool)
mask[:17] = False
mask[60] = False
mask[64] = False
Face = Face[mask,:]
num_features = 7
V = eigenvectors[:,1:num_features+1]  #* (eigenvalues[1:num_features+1]**2)
clustering_model = KMeans(n_clusters=10, init='k-means++', 
                          n_init=1000, max_iter=1000, tol=0.0001)
classes = clustering_model.fit_predict(V)
classes = classes.reshape((N_landmarks,2))
f_clusters = 20 * classes[:,0] + classes[:,0]
l_clusters = np.unique(f_clusters)
c_list = []
for c in l_clusters:
    c_list.append(np.argwhere(f_clusters == c).squeeze())
    
plt.figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')

colors = hsv(np.linspace(0, 1.0, len(c_list)))
for i in range(len(c_list)):
    indices = c_list[i]
    plt.scatter(Face[indices,0], -Face[indices,1], color=colors[i], linewidths=5)

In [ ]:
newV = V.reshape([N_landmarks,2,num_features]).sum(1)
Z = linkage(newV, method='ward')
fig = plt.figure(figsize=(30, 15))
dn = dendrogram(Z)

In [ ]:
class Clusters(dict):
    def _repr_html_(self):
        html = '<table style="border: 0;">'
        for c in self:
            hx = rgb2hex(colorConverter.to_rgb(c))
            html += '<tr style="border: 0;">' \
            '<td style="background-color: {0}; ' \
                       'border: 0;">' \
            '<code style="background-color: {0};">'.format(hx)
            html += c + '</code></td>'
            html += '<td style="border: 0"><code>' 
            html += repr(self[c]) + '</code>'
            html += '</td></tr>'
        
        html += '</table>'
        
        return html
    
def get_cluster_classes(den, label='ivl'):
    cluster_idxs = defaultdict(list)
    for c, pi in zip(den['color_list'], den['icoord']):
        for leg in pi[1:3]:
            i = (leg - 5.0) / 10.0
            if abs(i - int(i)) < 1e-5:
                cluster_idxs[c].append(int(i))
    
    cluster_classes = Clusters()
    for c, l in cluster_idxs.items():
        i_l = [den[label][i] for i in l]
        cluster_classes[c] = i_l
    
    return cluster_classes

get_cluster_classes(dn)

Ad hoc code to generate fixed hierachy, the values are chosen for there to be 4 layers, with the early ones having 5 to 6 clusters and the later stages to have 2 or 3.

In [ ]:
n = 4
threshold = list(np.linspace(0,2,4))
threshold  = [0] + list(np.logspace(-0.1,np.log(1.35),n))
threshold

In [ ]:
tree = []
for k in threshold:
    tree.append(fcluster(Z, k, criterion='distance'))

In [ ]:
shape = tree[0].shape[0]
arr = np.concatenate(tree).reshape([shape,len(tree)], order='F') - 1

In [ ]:
arr = np.repeat(arr, 2, axis=0)

for i in range(N_landmarks):
    arr[2*i,0] = 2*arr[2*i,0]
    arr[2*i+1,0] = 2*arr[2*i+1,0]+1

In [ ]:
for i in range(n+1):
    print(len(np.unique(arr[:,i])))

In [ ]:
arr

In [ ]:
np.save('new_tree', arr)